In [1]:
%autosave 0
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

# Téléchargez la liste de stop words en français
nltk.download('stopwords')

# Importez la liste de stop words en français
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_test = pd.read_excel("df_test.xlsx")
df_test.head(10)

,Client,Nombre_avis_publie,Note,Sentiment,Commentaire,Date_experience,Date_publication,Verification
0,MAXIME,1,1,Négatif,"Pneu commandé le 14 juillet, toujours pas de p...",2023-07-16,2023-08-04,1
1,Domy Comte,2,2,Négatif,Je commande un pneu de remorque en précisant l...,2023-07-23,2023-08-05,1
2,Francis Collette,3,3,Neutre,"3 étoiles pour le prix, mais mon avis reste mi...",2023-07-13,2023-08-03,1
3,Arlette,1,2,Négatif,Ma première expérience d'il y a 2 ou 3 ans ava...,2023-07-19,2023-08-03,1
4,Vitali,1,1,Négatif,Nul nul nul j'annule une commande au bout d'un...,2023-07-20,2023-08-04,1
5,eugenie,2,1,Négatif,Gros problème de livraison Mes pneus ne sont s...,2023-07-16,2023-08-05,1
6,Marc,2,1,Négatif,Site à fuire…!!!!!Commande passé le 7 juillet ...,2023-08-05,2023-08-05,0
7,Gavila Mathieu,4,2,Négatif,Allopneus nai pas les moins cher il y a d'autr...,2023-07-20,2023-08-04,1
8,Celou,2,1,Négatif,Je reçois 3 jours après ma commande les mauva...,2023-07-12,2023-08-01,1
9,Nico,1,1,Négatif,Nous avons passé commande le 20/07 pour une li...,2023-07-20,2023-08-03,1


In [3]:
# (a) Supprimer les caractères spéciaux, chiffres, valeurs manquantes et mettre le texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test.head(10)

C:\Users\micka\AppData\Local\Temp\ipykernel_6064\550896256.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()


,Client,Nombre_avis_publie,Note,Sentiment,Commentaire,Date_experience,Date_publication,Verification
0,MAXIME,1,1,Négatif,pneu command le juillet toujours pas de pneu ...,2023-07-16,2023-08-04,1
1,Domy Comte,2,2,Négatif,je commande un pneu de remorque en prcisant la...,2023-07-23,2023-08-05,1
2,Francis Collette,3,3,Neutre,toiles pour le prix mais mon avis reste mitig...,2023-07-13,2023-08-03,1
3,Arlette,1,2,Négatif,ma premire exprience dil y a ou ans avait t ...,2023-07-19,2023-08-03,1
4,Vitali,1,1,Négatif,nul nul nul jannule une commande au bout dune ...,2023-07-20,2023-08-04,1
5,eugenie,2,1,Négatif,gros problme de livraison mes pneus ne sont so...,2023-07-16,2023-08-05,1
6,Marc,2,1,Négatif,site fuirecommande pass le juillet je devai...,2023-08-05,2023-08-05,0
7,Gavila Mathieu,4,2,Négatif,allopneus nai pas les moins cher il y a dautre...,2023-07-20,2023-08-04,1
8,Celou,2,1,Négatif,je reois jours aprs ma commande les mauvais ...,2023-07-12,2023-08-01,1
9,Nico,1,1,Négatif,nous avons pass commande le pour une livraiso...,2023-07-20,2023-08-03,1


In [4]:
# (b) Division du DataFrame en ensembles d'entraînement et de test.
X_train, X_test, y_train, y_test = train_test_split(df_test['Commentaire'], df_test['Sentiment'], test_size=0.2, random_state=42)

In [5]:
# (c) Vectorisation du texte avec CountVectorizer.
vectorizer = TfidfVectorizer(stop_words=list(stop_words_french))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [6]:
# (e) Entraînement et évaluation des modèles
models = {
    'GradientBoostingClassifier': GradientBoostingClassifier(n_estimators=100, max_depth=3),
    'RandomForestClassifier': RandomForestClassifier(n_estimators=100, max_depth=3),
    'SVM': SVC(kernel='linear')
}

results = pd.DataFrame(columns=['Model', 'Precision', 'Recall', 'F1-Score'])

for model_name, model in models.items():
    model.fit(X_train_vectorized, y_train)
    y_pred = model.predict(X_test_vectorized)
    report = classification_report(y_test, y_pred, output_dict=True)
    results = results.append({
        'Model': model_name,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    }, ignore_index=True)

# Affichage des résultats dans un tableau
print(results)

C:\Users\micka\AppData\Local\Temp\ipykernel_6064\1741522094.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\micka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\micka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\micka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-def

                        Model  Precision    Recall  F1-Score
0  GradientBoostingClassifier   0.779645  0.805789  0.778918
1      RandomForestClassifier   0.624486  0.570003  0.417307
2                         SVM   0.840735  0.854136  0.844906


C:\Users\micka\AppData\Local\Temp\ipykernel_6064\1741522094.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
